### Imports

In [ ]:
from psycopg2.extras import execute_values, execute_batch
from reddit_stuff import backfill_submissions
from psycopg2.extras import Json
from pmaw import PushshiftAPI
from tqdm.auto import tqdm
import pandas as pd
import numpy as np 
from collections import Counter
from  itertools import chain
import datetime
import psycopg2
import requests
import json
import praw
import time
import re

pmaw_api = PushshiftAPI()

import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
%matplotlib inline

tqdm.pandas()

### Functions

In [ ]:
if "get_conn" in locals():
    del get_conn
    
def get_conn() -> psycopg2.connect:
    return psycopg2.connect(
        dbname="polygonio", 
        port="5433", 
        host="localhost", 
        password='rogerthat', 
        user="postgres"
    )

if "get_all_wsb_comments_analytics" in locals():
    del get_all_wsb_comments_analytics
    
def get_all_wsb_comments_analytics(start_str:str) -> pd.DataFrame:
    # start = datetime.datetime.strptime(start_str, "%Y-%m-%d").timestamp()
    ### Check if these are present in the database, and how many comments do each of these submissions have?
    with get_conn() as conn: 
        with conn.cursor() as cur: 
            cur.execute(f"SELECT created_utc::date as date, submission_id, comment_id, tickers FROM wsb_comments_analytics WHERE created_utc::date >= '{start_str}'::date;")
            df = pd.DataFrame(cur.fetchall(), columns=["date", "submission_id", "comment_id", "tickers"])
    return df

### Analyse

In [ ]:
df = get_all_wsb_comments_analytics('2021-01-01')
df.tickers.fillna(value=np.NaN, inplace=True)

In [ ]:
df.info()

In [ ]:
df2 = pd.DataFrame(df.loc[~pd.isna(df["tickers"]), ].groupby(["submission_id"])["tickers"], columns=["submission_id", "tickers_mentioned"])
df2.loc[:, "tickers_mentioned"] = df2["tickers_mentioned"].apply(lambda x: list(np.ravel(x)))
df2.loc[:, "tickers_mentioned"] = df2["tickers_mentioned"].apply(lambda x: list(chain.from_iterable(x) ))
df2["tickers_count"] = df2["tickers_mentioned"].progress_apply(lambda x: dict(Counter(chain(x))) if len(x) > 1 else dict(Counter(chain(*[x]))))
df2 = df2.drop(columns=["tickers_mentioned"])

In [ ]:
final_df = pd.merge(left=df2, right=df[["submission_id", "date"]], left_on="submission_id", right_on="submission_id", how="inner")
final_df = final_df.sort_values(by="date")
final_df = final_df.drop_duplicates(subset=["submission_id"])
final_df = final_df.reset_index(drop=True)

In [ ]:
final_df2_list = []
for sub_id in tqdm(final_df.submission_id.unique()): 
    date, tickers_count = final_df.loc[final_df["submission_id"] == sub_id, ["date", "tickers_count"]].values.tolist()[0]
    tickers_count = dict(sorted(tickers_count.items(), key=lambda item: item[1]))
    final_df2_list.append({"date": date, "submission_id": sub_id, "tickers_count": tickers_count})

final_df2 = pd.DataFrame.from_dict(final_df2_list)
final_df2.head()

In [ ]:
### Track a stock like gme across dates
ticker = "AMC"
final_df3 = pd.DataFrame(final_df2.groupby(["date"])["tickers_count"].apply(lambda x: pd.DataFrame(list(x)).sum() ))
final_df3 = final_df3.reset_index()
final_df3.columns = ["date", "ticker", "ticker_count"]
final_df3 = final_df3.set_index("date")
final_df3[final_df3["ticker"] == ticker.lower()].plot()